# Ecological Inference through Tsallis Regularized Optimal Transport (TROT)
This notebook presents the pipeline used in (cite our paper) to perform ecological inference on the Florida dataset.

You will first want to download the dataset from (url to the dataset)

In [13]:
import pandas as pd
import numpy as np
import pickle
import Distances as dist
from matplotlib import pyplot as plt
from matplotlib.pylab import savefig
from Florida_inference import CV_Local_Inference, Local_Inference
from Evaluation import KL

# Data Loading and Processing

In [14]:
FlData = pd.read_csv('Fl_Data.csv', usecols = ['District', 'County','Voters_Age', 'Voters_Gender', 'PID', 'vote08', 
                    'SR.WHI', 'SR.BLA', 'SR.HIS', 'SR.ASI', 'SR.NAT', 'SR.OTH'],nrows = 200000)

FlData = FlData.dropna()

Change gender values to numerical values

In [15]:
FlData['Voters_Gender'] = FlData['Voters_Gender'].map({'M': 1, 'F': 0})

Renormalize the age so that it takes values between 0 and 1

In [16]:
FlData['Voters_Age'] = ((FlData['Voters_Age'] -
                         FlData['Voters_Age'].min()) /
                        (FlData['Voters_Age'].max() -
                         FlData['Voters_Age'].min()))


One-hot party subscriptions (PID)

In [17]:
#Get one hot encoding of column PID
one_hot = pd.get_dummies(FlData['PID'])
# Drop column PID as it is now encoded
FlData = FlData.drop('PID', axis=1)
# Join the encoded df
FlData = FlData.join(one_hot)
# Rename the new columns
FlData.rename(columns={0: 'Other', 1: 'Democrat', 2: 'Republican'},
              inplace=True)

In [18]:
FlData.describe()

,District,County,Voters_Age,Voters_Gender,vote08,SR.WHI,SR.BLA,SR.HIS,SR.ASI,SR.NAT,SR.OTH,Other,Democrat,Republican
count,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000,181846.000000
mean,6.543911,26.616104,0.377654,0.456672,0.749695,0.768194,0.117973,0.049888,0.021683,0.003745,0.041035,0.226807,0.473038,0.300155
std,1.905905,21.417582,0.237857,0.498121,0.433190,0.421987,0.322578,0.217715,0.145647,0.061081,0.198371,0.418768,0.499274,0.458326
min,3.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,12.000000,0.162500,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,12.000000,0.375000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,58.000000,0.562500,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,9.000000,58.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Compute Marginals and Joint Distributions

Create a county dictionnary

In [19]:
Voters_By_County = {}
all_counties = FlData.County.unique()
for county in all_counties:
    Voters_By_County[county] = FlData[FlData['County'] == county]

Compute the ground truth joint distribution

In [20]:
J = {}
for county in all_counties:
    J[county] = np.zeros((6, 3))

    J[county][0,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.WHI']==1)].shape[0]
    J[county][0,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.WHI']==1)].shape[0]
    J[county][0,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.WHI']==1)].shape[0]

    J[county][1,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.BLA']==1)].shape[0]
    J[county][1,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.BLA']==1)].shape[0]
    J[county][1,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.BLA']==1)].shape[0]

    J[county][2,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.HIS']==1)].shape[0]
    J[county][2,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.HIS']==1)].shape[0]
    J[county][2,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.HIS']==1)].shape[0]

    J[county][3,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.ASI']==1)].shape[0]
    J[county][3,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.ASI']==1)].shape[0]
    J[county][3,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.ASI']==1)].shape[0]

    J[county][4,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) &(Voters_By_County[county]['SR.NAT']==1)].shape[0]
    J[county][4,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.NAT']==1)].shape[0]
    J[county][4,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.NAT']==1)].shape[0]

    J[county][5,0] = Voters_By_County[county].loc[(Voters_By_County[county]['Other'] ==1) & (Voters_By_County[county]['SR.OTH']==1)].shape[0]
    J[county][5,1] = Voters_By_County[county].loc[(Voters_By_County[county]['Democrat'] ==1) & (Voters_By_County[county]['SR.OTH']==1)].shape[0]
    J[county][5,2] = Voters_By_County[county].loc[(Voters_By_County[county]['Republican'] ==1) & (Voters_By_County[county]['SR.OTH']==1)].shape[0]

    J[county] /= J[county].sum()

In [21]:
print(J[12])

[[ 0.14002316  0.31636083  0.24990753]
 [ 0.01778736  0.14102832  0.00533942]
 [ 0.01595394  0.02251564  0.0117403 ]
 [ 0.01128194  0.01120957  0.00474437]
 [ 0.00099712  0.0018495   0.00104537]
 [ 0.01933933  0.02032037  0.00855594]]


Compute the party marginals

In [22]:
Party_Marginals = {}
parties = ['Other', 'Democrat', 'Republican']
for county in all_counties:
    Party_Marginals[county] = pd.Series([J[county][:, i].sum()
                                        for i in np.arange(3)])
    Party_Marginals[county].index = parties

Compute the ethnicity marginals

In [23]:
Ethnicity_Marginals = {}
ethnies = ['SR.WHI', 'SR.BLA', 'SR.HIS', 'SR.ASI', 'SR.NAT', 'SR.OTH']
for county in all_counties:
    Ethnicity_Marginals[county] = pd.Series([J[county][i, :].sum()
                                             for i in np.arange(6)])
    Ethnicity_Marginals[county].index = ethnies

# Compute the cost matrix
Using only age, gender, and 2008 vote or abstention

In [24]:
features = ['Voters_Age', 'Voters_Gender', 'vote08']
e_len, p_len = len(ethnies), len(parties)
M = np.zeros((e_len, p_len))
for i, e in enumerate(ethnies):
    data_e = FlData[FlData[e] == 1.0]
    average_by_e = data_e[features].mean(axis=0)
    for j, p in enumerate(parties):
        data_p = FlData[FlData[p] == 1.0]
        average_by_p = data_p[features].mean(axis=0)

        M[i, j] = np.array(dist.dist_2(average_by_e, average_by_p))

# Start the inference

Use a specific county or district to select the best parameters

In [25]:
CV_counties = FlData[FlData['District'] == 3].County.unique()

Find the best parameters

In [26]:
q = np.arange(0.5, 2.1, 0.1)
l = [0.01, 0.1, 1., 10., 100., 1000.] 

best_score, best_q, best_l = CV_Local_Inference(Voters_By_County, M, J, Ethnicity_Marginals, Party_Marginals,
                   CV_counties,q,l)

q: 0.50, lambda: 0.0100, KL: 0.1245, STD: 0
q: 0.50, lambda: 0.1000, KL: 0.1233, STD: 0
q: 0.50, lambda: 1.0000, KL: 0.1124, STD: 0
q: 0.50, lambda: 10.0000, KL: 0.0826, STD: 0
q: 0.50, lambda: 100.0000, KL: 0.2973, STD: 0
q: 0.50, lambda: 1000.0000, KL: 0.3971, STD: 0
q: 0.60, lambda: 0.0100, KL: 0.1184, STD: 0
q: 0.60, lambda: 0.1000, KL: 0.1171, STD: 0
q: 0.60, lambda: 1.0000, KL: 0.1052, STD: 0
q: 0.60, lambda: 10.0000, KL: 0.08138, STD: 0
q: 0.60, lambda: 100.0000, KL: 0.3137, STD: 0
q: 0.60, lambda: 1000.0000, KL: 0.4101, STD: 0
q: 0.70, lambda: 0.0100, KL: 0.1122, STD: 0
q: 0.70, lambda: 0.1000, KL: 0.1107, STD: 0
q: 0.70, lambda: 1.0000, KL: 0.09715, STD: 0
q: 0.70, lambda: 10.0000, KL: 0.08489, STD: 0
q: 0.70, lambda: 100.0000, KL: 0.33, STD: 0
q: 0.70, lambda: 1000.0000, KL: 0.4013, STD: 0
q: 0.80, lambda: 0.0100, KL: 0.1059, STD: 0
q: 0.80, lambda: 0.1000, KL: 0.1041, STD: 0
q: 0.80, lambda: 1.0000, KL: 0.08853, STD: 0
q: 0.80, lambda: 10.0000, KL: 0.09611, STD: 0
q: 0.80, l

Use selected parameters on the rest of the dataset

In [27]:
J_inferred = Local_Inference(Voters_By_County, M, J, Ethnicity_Marginals, Party_Marginals, all_counties, best_q, best_l)
kl, std = KL(J, J_inferred, all_counties, save_to_file=False, compute_abs_err=True)

Absolute error 0.0109869281715  +  0.00212937107972


# Plot the results

In [28]:
diag = np.linspace(-0.1, 1.0, 100)

# pickle results
f = open('joints_gallup.pkl', 'rb')
J_true, J = pickle.load(f)

f = open('baseline.pkl', 'rb')
J_baseline = pickle.load(f)

j_true, j, j_baseline = [], [], []
for c in all_counties:
    j_true.append(np.array(J_true[c]).flatten())
    j.append(np.array(J_inferred[c]).flatten())
    j_baseline.append(np.array(J_baseline[c]).flatten())

j_true = np.array(j_true).flatten()
j = np.array(j).flatten()
j_baseline = np.array(j_baseline).flatten()

Plot the correlation between the ground truth for the joint distribution and the infered distribution (the closer to the $x = y$ diagonal axis, the better

In [29]:
plt.figure()
plt.scatter(j_true, j, alpha=0.5)
plt.xlabel('Ground truth')
plt.ylabel('TROT (RBF)')
plt.plot(diag, diag, 'r--')

plt.show()

Plot the distribution of the error (the more packed around the origin of the $x$-axis, the better)

In [30]:
plt.figure()
bins = np.arange(-.3, .6, 0.01)
plt.hist(j_true - j, bins=bins, alpha=0.5, label='TROT')
plt.hist(j_true - j_baseline, bins=bins, alpha=0.5, label='Florida-average')
plt.legend()
plt.xlabel('Difference between inference and ground truth')

plt.show()

# Survey-based ecological inference
Same pipeline, but using a cost matrix computed thanks to the 2013 Gallup survey. (http://www.gallup.com/poll/160373/democrats-racially-diverse-republicans-mostly-white.aspx)

We assume that Gallup's Other = {Native, Other}

The cost matrix M is computed as $1-p_{ij}$, where $p_{ij}$ is the proportion of people registered to party $j$ belonging to group $i$.

In [31]:
M_sur = np.array([
               [.38, .26, .35],
               [.29, .64, .05],
               [.50, .32, .13],
               [.46, .36, .17],
               [.49, .32, .18],
               [.49, .32, .18]
               ])
M_sur = (1. - M_sur)

Once again, find the best parameters

In [32]:
best_score, best_q, best_l = CV_Local_Inference(Voters_By_County, M_sur, J, Ethnicity_Marginals, Party_Marginals,
                   CV_counties,q,l)

q: 0.50, lambda: 0.0100, KL: 0.1569, STD: 0
q: 0.50, lambda: 0.1000, KL: 0.1538, STD: 0
q: 0.50, lambda: 1.0000, KL: 0.1238, STD: 0
q: 0.50, lambda: 10.0000, KL: 0.008533, STD: 0
q: 0.50, lambda: 100.0000, KL: 0.0585, STD: 0
q: 0.50, lambda: 1000.0000, KL: 0.9554, STD: 0
q: 0.60, lambda: 0.0100, KL: 0.1488, STD: 0
q: 0.60, lambda: 0.1000, KL: 0.1453, STD: 0
q: 0.60, lambda: 1.0000, KL: 0.1135, STD: 0
q: 0.60, lambda: 10.0000, KL: 0.003816, STD: 0
q: 0.60, lambda: 100.0000, KL: 0.07131, STD: 0
q: 0.60, lambda: 1000.0000, KL: 0.8529, STD: 0
q: 0.70, lambda: 0.0100, KL: 0.14, STD: 0
q: 0.70, lambda: 0.1000, KL: 0.1362, STD: 0
q: 0.70, lambda: 1.0000, KL: 0.1017, STD: 0
q: 0.70, lambda: 10.0000, KL: 0.001322, STD: 0
q: 0.70, lambda: 100.0000, KL: 0.08656, STD: 0
q: 0.70, lambda: 1000.0000, KL: 0.6588, STD: 0
q: 0.80, lambda: 0.0100, KL: 0.1308, STD: 0
q: 0.80, lambda: 0.1000, KL: 0.1266, STD: 0
q: 0.80, lambda: 1.0000, KL: 0.08878, STD: 0
q: 0.80, lambda: 10.0000, KL: 0.00295, STD: 0
q: 0.

Using these parameters, run the inference on the rest of the dataset

In [33]:
J_sur = Local_Inference(Voters_By_County, M_sur, J, Ethnicity_Marginals, Party_Marginals, all_counties, best_q, best_l)
kl, std = KL(J, J_sur, all_counties, save_to_file=False, compute_abs_err=True)

Absolute error 0.00449674812651  +  0.00448982091371


Plot correlation with ground truth

In [34]:
j_sur = []
for c in all_counties:
    j_sur.append(np.array(J_sur[c]).flatten())

j_sur = np.array(j_sur).flatten()

plt.figure()
plt.scatter(j_true, j_sur, alpha=0.5)
plt.xlabel('Ground truth')
plt.ylabel('TROT (survey)')
plt.plot(diag, diag, 'r--')

plt.show()
    

Plot error distribution (compared with Florida average)

In [35]:
plt.figure()
bins = np.arange(-.3, .6, 0.01)
plt.hist(j_true - j_sur, bins=bins, alpha=0.5, label='TROT (survey)')
plt.hist(j_true - j_baseline, bins=bins, alpha=0.5, label='Florida-average')
plt.legend()
plt.xlabel('Difference between inference and ground truth')

plt.show()